In [ ]:
%%writefile catch.py

# Import

import os
import sys
from datetime import datetime
import time
import requests
import re

import pandas as pd

import chromedriver_autoinstaller
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException


def WC_Catch(corp=str) :
    
    # headless mode

    option = webdriver.ChromeOptions()
    option.add_argument('--headless')        # Head-less 설정
    option.add_argument('--no-sandbox')
    option.add_argument('--disable-dev-shm-usage')

    service = Service(executable_path='chromedriver.exe')
    browser = webdriver.Chrome(service=service, options=option)

    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')

    url = r"https://www.catch.co.kr/"

    browser.maximize_window()
    print(browser.get_window_size())

    browser.get(url)



    # 로그인

    from accounts import ct

    ID = ct.ID
    PW = ct.PW



    browser.find_element(By.CSS_SELECTOR, "a.login").click()

    browser.find_element(By.CSS_SELECTOR, "input#id_login").send_keys(ID)

    browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(PW)

    browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(Keys.ENTER)

    time.sleep(2)



    query = browser.find_element(By.CSS_SELECTOR, "input")

    query.send_keys(corp)

    time.sleep(1)

    query.send_keys(Keys.ENTER)

    time.sleep(2)



    ### 팝업 뜨면 끄기 (예외 처리)

    try : 
        browser.find_element(By.CSS_SELECTOR, 'button.today > span').click()
        
    except : 
        pass

    time.sleep(2)



    # 검색한 기업(최상위 노출) 클릭 
    browser.find_element(By.CSS_SELECTOR, "p.name > a").click()

    time.sleep(1)

    # '현직자 리뷰' 탭 클릭
    browser.find_element(By.XPATH, r'//*[@id="Contents"]/div[1]/div[1]/div[2]/ul/li[4]/a').click()

    time.sleep(1)

    # 핵심 리뷰만 조회 - 정규직/현직자가 3년 이내 작성한 리뷰
    browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > div > a').click()

    time.sleep(1)





    #########################################################################

    time.sleep(1)

    # Review Crawler

    n_btn = browser.find_element(By.CSS_SELECTOR, 'a.ico.next')

    time.sleep(1)

    good_lst = None
    bad_lst = None


    good_lst = []
    bad_lst = []


    pre_page = 0


    while True : 

        time.sleep(1)

        # 현재 페이지의 숫자
        crt_btn = browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > p > a.selected')
        crt_page = int(crt_btn.text)

        # 다음 버튼 눌러도 같은 페이지의 경우 (마지막 페이지)
        if crt_page == pre_page:
            break

        try:
            for p in range(1, 5) : 

                grv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(4)").text
                brv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(5)").text

                grv_clean = grv.replace(grv[:5], "").replace('\n', ' ') 
                brv_clean = brv.replace(brv[:4], "").replace('\n', ' ') 

                good_lst.append(grv_clean)
                bad_lst.append(brv_clean)
        
        except NoSuchElementException : 
            break

        time.sleep(2)

        # WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(4)")))


    # 다음 페이지로 이동하기 위한 버튼을 찾기.
        try:
            # 이전페이지의 번호를 갱신
            pre_page = crt_page

            # 다음 버튼 클릭
            n_btn.click()

            # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
            time.sleep(1)

        except:
            break



    browser.close()


    # list to dataframe

    good_lst = list(set(good_lst))
    bad_lst = list(set(bad_lst))

    good_df = pd.DataFrame({"긍정적 평가" : good_lst})
    bad_df = pd.DataFrame({"부정적 평가" : bad_lst})

    pros_cons = good_df.join(bad_df)
    pros_cons


In [1]:
from catch import WC_Catch

In [2]:
WC_Catch('삼성전자')

{'width': 800, 'height': 600}


,긍정적 평가,부정적 평가
0,회사와 함께 성장이 가능하다. 다양한 업무를 경험할 수 있다.,전반적으로 업무 강도가 있는 편으로 업무에 따라서 퇴근시간이 늦어지는 경우가 있을 ...
1,"근무환경, 복지가 매우매우 좋음. 조직문화도 빠르게 변화/발전하는 중",포괄수당으로 인한 20시간 근무. 부서간 이동에 제약 등
2,"적당한 연봉, 다양한 식사 제공, 자기자신만의 능력을 발휘할 수 있는 환경이 조성되...","경쟁, 포괄임금제, 경영진 MBO 잦은 조직개편 및 강제전배(직원커리어 무시)"
3,휴가 맘대로 씀. 시스템이 잘 되어있어서 하나의 실수가 치명적으로 다가오진 않음,과도하게 많은 의사결정단계로 인한 비효율. 의사결정권자의 단기지향적인 사고방식
4,"함께 일하는 동료들이 유하다. 셔틀버스, 삼시세끼 챙겨 먹을 수 있는 식당, 사내병...",전문적인 지식을 요하는 부분이 많은데 교육의 기회가 다소 부족함.
...,...,...
126,교통도 매우 편하고 직원들 편의시설 및 복지 시설도 편리하다,"연봉 인상률이 낮으며, 부서에 따라 워라벨 보장 안됨. 잡다한 일이 굉장히 많음"
127,복리후생과 교육제도가 잘 되있고..여가문화도 제공한다.,남자비율이 너무 높은 것같다. 아직은 아쉬운 점이 딱히없다.
128,관리의 삼성 기업시스템이 무엇인지 알 수 있음. 많이 배울 수 있다,"회사가 경기 남부권이라, 서울에서 출퇴근 하기에는 살짝 부담스럽습니다. 또한 정년이..."
129,직원 복지 정책 우선이다 능력 우선 주의 항상 최선을 다 하면 좋은 결과 있다,타 대기업 대비 복지가 안좋음 직급 올라갈수록 세금을 많이 뜯어가서 성과급이 별로 ...
